---------------------------------------DATA SCRAPPING USING SELENIUM-------------------------------

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
driver_path ='C:\Program Files (x86)\chromedriver-win64\chromedriver.exe'
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
#we have scrapped the following youtube channel as it can contain many irrelivant/spam comments.
url = 'https://www.youtube.com/@DuckyBhai/featured'
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [ ]:
videos_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/yt-tab-group-shape/div[1]/yt-tab-shape[2]/div[1]')
videos_button.click()

In [ ]:
for _ in range(5):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)

videos = driver.find_elements(By.CLASS_NAME, "style-scope ytd-rich-grid-media")
link_array = []
for video in videos:
    link = video.find_element(By.XPATH, './/*[@id="video-title-link"]').get_attribute('href')
    link_array.append(link)


driver.quit()

In [ ]:
for index, link in enumerate(link_array):
    print(f"Index {index}: {link}")


In [ ]:
i=0
for link in link_array:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(link)
    time.sleep(2)


    for _ in range(5):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)

    comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    arr = []
    for comment in comments:
        arr.append(comment.text)

    i=i+1
    print(i)

    df = pd.DataFrame({'Comments': arr})

    df.to_csv('youtube_comments_scrapped_data.csv', mode='a',header=False, index=False)

    driver.close()


-------------------------------------------DATA WRANLING---------------------

In [ ]:
import pandas as pd
column_names = ['Comments']
raw_data= pd.read_csv('youtube_comments_scrapped_data.csv', header=None, names=column_names)
raw_data

,Comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...
1,Who is proud to be Muslim ?
2,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️
4,Who is proud to be a Muslim
...,...
12887,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS


dropping null rows

In [ ]:
raw_data= raw_data.dropna()
raw_data

,Comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...
1,Who is proud to be Muslim ?
2,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️
4,Who is proud to be a Muslim
...,...
12887,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS


In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install translate
!pip install langdetect
!pip install emoji
!pip install pyspellchecker

-------------------------removing emoji--------------

In [ ]:
import emoji
import re

def remove_emojis(text):
    text_no_emojis = text.encode('ascii', 'ignore').decode('ascii')
    text_no_emojis = re.sub(r'[^a-zA-Z0-9\s]', '', text_no_emojis)
    return text_no_emojis

raw_data['remove_emoji_comments'] = raw_data['Comments'].apply(remove_emojis)
raw_data[['remove_emoji_comments']]

<ipython-input-11-2aecb51c8b9f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['remove_emoji_comments'] = raw_data['Comments'].apply(remove_emojis)


,remove_emoji_comments
0,I bought MECLAY LONDON Thick Dense Shampoo at...
1,Who is proud to be Muslim
2,Proud to be a Muslim
3,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim
...,...
12887,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS


In [ ]:
raw_data['remove_emoji_comments']=raw_data['remove_emoji_comments'].dropna()
raw_data

<ipython-input-12-45c12a183b78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['remove_emoji_comments']=raw_data['remove_emoji_comments'].dropna()


,Comments,remove_emoji_comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...,I bought MECLAY LONDON Thick Dense Shampoo at...
1,Who is proud to be Muslim ?,Who is proud to be Muslim
2,Proud to be a Muslim,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim,Who is proud to be a Muslim
...,...,...
12887,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK \nGOOD TO SEE YOU ALWAYS


---------------------------------------REMOVING EXTRA SPACES AND SPECIAL CHARACTERS-----------------

In [ ]:
import re
def clean_sentence(sentence):
    if isinstance(sentence, str):
        cleaned_sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence).strip()
        return cleaned_sentence
    else:
        return ''

raw_data['clean_comments'] =raw_data['remove_emoji_comments'].apply(lambda x: clean_sentence(x))
raw_data

<ipython-input-13-1446e3a30efe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['clean_comments'] =raw_data['remove_emoji_comments'].apply(lambda x: clean_sentence(x))


,Comments,remove_emoji_comments,clean_comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...,I bought MECLAY LONDON Thick Dense Shampoo at...,I bought MECLAY LONDON Thick Dense Shampoo at ...
1,Who is proud to be Muslim ?,Who is proud to be Muslim,Who is proud to be Muslim
2,Proud to be a Muslim,Proud to be a Muslim,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️,PROUD TO BE A MUSLIM,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim,Who is proud to be a Muslim,Who is proud to be a Muslim
...,...,...,...
12887,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull,His wife is cute n happy soull,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK GOOD TO SEE YOU ALWAYS


--------------------------REMOVING PUNCTUATION---------------

In [ ]:
import string
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    text_without_punct = text.translate(translator)
    return text_without_punct

raw_data['clean_comments'] = raw_data['clean_comments'].apply(lambda x: clean_sentence(x))
raw_data

<ipython-input-14-f7448398f9a9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['clean_comments'] = raw_data['clean_comments'].apply(lambda x: clean_sentence(x))


,Comments,remove_emoji_comments,clean_comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...,I bought MECLAY LONDON Thick Dense Shampoo at...,I bought MECLAY LONDON Thick Dense Shampoo at ...
1,Who is proud to be Muslim ?,Who is proud to be Muslim,Who is proud to be Muslim
2,Proud to be a Muslim,Proud to be a Muslim,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️,PROUD TO BE A MUSLIM,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim,Who is proud to be a Muslim,Who is proud to be a Muslim
...,...,...,...
12887,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull,His wife is cute n happy soull,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK GOOD TO SEE YOU ALWAYS


----------------------------TRANSLATING ALL MAJOR/MINOR COMMENTS INTO A SINGLE LANGUAGE--------

In [ ]:
from langdetect import detect
from googletrans import Translator
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def translate_to_english(word):
    if word is None or not word.strip():
        return None

    try:
        translator = Translator()
        translation = translator.translate(word, dest='en')
        return translation.text

    except Exception as e:
        print(f"Error translating '{word}': {e}")
        return word

num_threads = 8
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    translated_comments = list(executor.map(translate_to_english, raw_data['clean_comments']))

raw_data['translated_comments'] = translated_comments

Streaming output truncated to the last 5000 lines.
Error translating 'Congratulations for 1M followers Wife ki': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Who Proud to be Muslim': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Congratulations Aroob bhabi for 1M': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Anxiously waiting for your vlog getting out of every tension after watching ur vlog': 'Translator' object has no attribute 'raise_Exception'
Error translating 'proud to be Muslim': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Lets appreciate that this man never disappointed us with his video': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Lets vote for DUCKY BHAI saying yes to mama jaan for 24 hours': 'Translator' object has no attribute 'raise_Exception'
Error translating 'Ducky bhai is literally a Mood Changer for me': 'Translator' object

<ipython-input-15-5507ab7b06e7>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['translated_comments'] = translated_comments


In [ ]:
raw_data['translated_comments'].to_csv('youtube translated_comments_using_threads.csv', mode='a',header='translated_comments', index=False)

In [ ]:
import pandas as pd
translation = pd.read_csv('youtube translated_comments_using_threads.csv')
translation

,translated_comments
0,I bought MECLAY LONDON Thick Dense Shampoo at ...
1,Who is proud to be Muslim
2,Proud to be a Muslim
3,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim
...,...
12876,May Allah protect u always ameen safe journey
12877,His wife is cute n happy soull
12878,love you so much shazada veer love from jhelum...
12879,GREAT WORK GOOD TO SEE YOU ALWAYS


In [ ]:
missing_values = translation['translated_comments'].isnull().sum()
print(f'Number of missing values: {missing_values}')
translation['translated_comments'].fillna('', inplace=True)

Number of missing values: 0


In [ ]:
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.7 MB/s eta 0:00:00


In [ ]:
from spellchecker import SpellChecker
def correct_spelling(text):
    if pd.notna(text):
        spell = SpellChecker()
        words = text.split()
        misspelled = spell.unknown(words)

        corrected_text = []
        for word in words:
            correction = spell.correction(word)
            if correction is not None:
                corrected_text.append(correction)
            else:
                corrected_text.append(word)

        corrected_sentence = ' '.join(corrected_text)
        return corrected_sentence
    else:
        return text



translation['correct_spelling_translated'] = translation['translated_comments'].apply(lambda x: correct_spelling(x))

In [ ]:
translation.head()

,translated_comments,correct_spelling_translated
0,I bought MECLAY LONDON Thick Dense Shampoo at ...,I bought delay LONDON Thick Dense Shampoo at a...
1,Who is proud to be Muslim,Who is proud to be Muslim
2,Proud to be a Muslim,Proud to be a Muslim
3,PROUD TO BE A MUSLIM,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim,Who is proud to be a Muslim
...,...,...
12876,May Allah protect u always ameen safe journey,May Allah protect u always amen safe journey
12877,His wife is cute n happy soull,His wife is cute i happy soul
12878,love you so much shazada veer love from jhelum...,love you so much shazada veer love from helium...
12879,GREAT WORK GOOD TO SEE YOU ALWAYS,GREAT WORK GOOD TO SEE YOU ALWAYS


In [ ]:
raw_data

,Comments,remove_emoji_comments,clean_comments,translated_comments
0,I bought MECLAY LONDON Thick & Dense Shampoo a...,I bought MECLAY LONDON Thick Dense Shampoo at...,I bought MECLAY LONDON Thick Dense Shampoo at ...,I bought MECLAY LONDON Thick Dense Shampoo at ...
1,Who is proud to be Muslim ?,Who is proud to be Muslim,Who is proud to be Muslim,Who is proud to be Muslim
2,Proud to be a Muslim,Proud to be a Muslim,Proud to be a Muslim,Proud to be a Muslim
3,PROUD TO BE A MUSLIM ️,PROUD TO BE A MUSLIM,PROUD TO BE A MUSLIM,PROUD TO BE A MUSLIM
4,Who is proud to be a Muslim,Who is proud to be a Muslim,Who is proud to be a Muslim,Who is proud to be a Muslim
...,...,...,...,...
12887,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey,May Allah protect u always ameen safe journey
12888,His wife is cute n happy soull,His wife is cute n happy soull,His wife is cute n happy soull,His wife is cute n happy soull
12889,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...,love you so much shazada veer love from jhelum...
12890,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK \nGOOD TO SEE YOU ALWAYS,GREAT WORK GOOD TO SEE YOU ALWAYS,GREAT WORK GOOD TO SEE YOU ALWAYS


In [ ]:
missing_values = translation['translated_comments'].isnull().sum()
print(f'Number of missing values: {missing_values}')

Number of missing values: 0


-----------------------converting highcase words into lowercase comments-------------

In [ ]:
import re
clean = pd.DataFrame()
clean['clean_comments'] = translation['translated_comments'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x)))
clean['clean_comments'] = clean['clean_comments'].apply(lambda x: ' '.join(x.split()))
clean['clean_comments']=clean['clean_comments'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
clean

,clean_comments
0,i bought meclay london thick dense shampoo at ...
1,who is proud to be muslim
2,proud to be a muslim
3,proud to be a muslim
4,who is proud to be a muslim
...,...
12876,may allah protect u always ameen safe journey
12877,his wife is cute n happy soull
12878,love you so much shazada veer love from jhelum...
12879,great work good to see you always


In [ ]:
clean['clean_comments'].to_csv('input_file_for_labelling.csv', mode='a',header='comments', index=False)

In [ ]:
import pandas as pd

df_count = pd.read_csv('input_file_for_labelling.csv')
comment_counts = df_count['clean_comments'].value_counts()
comment_counts_df = pd.DataFrame({
    'Comment': comment_counts.index,
    'Count': comment_counts.values
})
comment_counts_df


,Comment,Count
0,proud to be a muslim,444
1,who is proud to be a muslim,353
2,proud to be muslim,293
3,who is proud to be muslim,184
4,lets vote for ignoring aroob for hours,148
...,...,...
8724,the time when azlaan opened the quran bookwas ...,1
8725,duckys end when he hugged shahveer while cryin...,1
8726,so much fun he had,1
8727,the expressions of duck bhai at the last,1


In [ ]:
comment_counts_df.to_csv('comment_count.csv', mode='a',header='comments', index=False)

------------------------labelling spam comments---------

In [ ]:
with open('final_spam_dictionaryy.txt', 'r') as file:
    file_content = file.read()

print(file_content)
print(len(file_content))

proud muslim,ummat,namaz,hazrat muhammad,hazrat muhammad saw,azaan,quraan,quranummah e,ummah,click link,link,ummate muslima,god,ummet,saww,muslims,rasool,proud,ummati,muslimp,stand palestine,palestine,loves quran,quran,discounted prices,daraz,muslimmmm,ramzan,ramzan mubarak,free,prize,click,check out,earn money,guarantee,limited time,act now,credit card,cash,discount,make money fast,100 satisfied,ignoring aroob,income,credit,refund,winner,unbelievable,extra cash,double your,increase sales,risk-free,trial,lowest price,money back,no catch,no fees,no hidden,remove,social security,spam,winner,unsubscribe,online pharmacy,gambling,adult,porn,dating,meet singles,lose weight,meet girls,earn per week,home based,work from home,income,easy money,exclusive deal,lifetime,urgent response,lowest price,hidden charges,no cost,no credit check,no hidden fees,no interest,no investment,notspam,one hundred percent,this is not a scam,this is not junk,unsubscribe anytime,winner,work from home,you are a winner

In [ ]:
import pandas as pd

input = pd.read_csv('input_file_for_labelling.csv')
with open('final_spam_dictionaryy.txt', 'r') as file:
    spam_words = set(file.read().split(','))

def is_spam(comment):
    if isinstance(comment, str):
        words = set(comment.split())
        return any(word in spam_words for word in words)
    else:
        return False

input['label'] = input['clean_comments'].apply(lambda x: 1 if pd.notna(x) and any(word in x for word in spam_words) else 0)


In [ ]:
input

,clean_comments,label
0,i bought meclay london thick dense shampoo at ...,1
1,who is proud to be muslim,1
2,proud to be a muslim,1
3,proud to be a muslim,1
4,who is proud to be a muslim,1
...,...,...
12876,may allah protect u always ameen safe journey,0
12877,his wife is cute n happy soull,0
12878,love you so much shazada veer love from jhelum...,0
12879,great work good to see you always,0


In [ ]:
input.to_csv('youtube_labelled_comments.csv', mode='a',header='comments', index=False)

In [ ]:
process=pd.read_csv('youtube_labelled_comments.csv')

In [ ]:
process

,clean_comments,label
0,i bought meclay london thick dense shampoo at ...,1
1,who is proud to be muslim,1
2,proud to be a muslim,1
3,proud to be a muslim,1
4,who is proud to be a muslim,1
...,...,...
12876,may allah protect u always ameen safe journey,0
12877,his wife is cute n happy soull,0
12878,love you so much shazada veer love from jhelum...,0
12879,great work good to see you always,0


-----------------------------tokenizes comments-----------------------------------

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
process=pd.read_csv('youtube_labelled_comments.csv')
process['clean_comments'] = process['clean_comments'].apply(lambda x: word_tokenize(str(x)))
process

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,clean_comments,label
0,"[i, bought, meclay, london, thick, dense, sham...",1
1,"[who, is, proud, to, be, muslim]",1
2,"[proud, to, be, a, muslim]",1
3,"[proud, to, be, a, muslim]",1
4,"[who, is, proud, to, be, a, muslim]",1
...,...,...
12876,"[may, allah, protect, u, always, ameen, safe, ...",0
12877,"[his, wife, is, cute, n, happy, soull]",0
12878,"[love, you, so, much, shazada, veer, love, fro...",0
12879,"[great, work, good, to, see, you, always]",0


-------------------------------REMOVING STOPWORDS----------------

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
process['clean_comments']  =process['clean_comments'] .apply(lambda x: [word for word in x if word.lower() not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
process

,clean_comments,label
0,"[bought, meclay, london, thick, dense, shampoo...",1
1,"[proud, muslim]",1
2,"[proud, muslim]",1
3,"[proud, muslim]",1
4,"[proud, muslim]",1
...,...,...
12876,"[may, allah, protect, u, always, ameen, safe, ...",0
12877,"[wife, cute, n, happy, soull]",0
12878,"[love, much, shazada, veer, love, jhelum, punjab]",0
12879,"[great, work, good, see, always]",0


In [ ]:
process['clean_comments'] = process['clean_comments'].apply(lambda x: ' '.join(x))
process

,clean_comments,label
0,bought meclay london thick dense shampoo amazi...,1
1,proud muslim,1
2,proud muslim,1
3,proud muslim,1
4,proud muslim,1
...,...,...
12876,may allah protect u always ameen safe journey,0
12877,wife cute n happy soull,0
12878,love much shazada veer love jhelum punjab,0
12879,great work good see always,0


In [ ]:
process.to_csv('stopwords_filtered.csv', mode='a',header='stopwords_filtered', index=False)

-------------------------------------FEATURE EXTRACTION------------------------

--------------------------------------------TDIDF------------------------------------------------

In [15]:
extraction=pd.read_csv('stopwords_filtered.csv')
extraction

,clean_comments,label
0,bought meclay london thick dense shampoo amazi...,1
1,proud muslim,1
2,proud muslim,1
3,proud muslim,1
4,proud muslim,1
...,...,...
12876,may allah protect u always ameen safe journey,0
12877,wife cute n happy soull,0
12878,love much shazada veer love jhelum punjab,0
12879,great work good see always,0


In [ ]:
!pip install gensim
!pip install scikit-learn

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('stopwords_filtered.csv')
df['clean_comments'] = df['clean_comments'].fillna('')

tfidf_feature_names = None
tfidf_df = pd.DataFrame()
vectorizer = TfidfVectorizer(analyzer='word')
tfidf_matrix = vectorizer.fit_transform(df['clean_comments'])
tfidf_feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)
num_features = len(tfidf_feature_names)
print("Number of features in TF-IDF matrix:", num_features)
tfidf_df

Number of features in TF-IDF matrix: 8354


,aa,aaa,aab,aabadh,aadhi,aaega,aagai,aagaya,aage,aagya,...,zipline,ziyarat,zndagi,zoo,zoomed,zuhr,zulkifarwho,zurriyatihi,zyada,zyda
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_df.to_csv('tfidf_df.csv', mode='a',header=False , index=False)

-------------------------------------------------------MACHINE LEARNING-------------------------------------------------------

------------------------------------------------------- Naive Bayes classifier-------------------------------------------------------

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['label'], test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

predictions = classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
confusion_mat = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("\nConfusion Matrix:")
print(confusion_mat)
print("\nClassification Report:")
print(classification_rep)


Accuracy: 0.9662398137369034

Confusion Matrix:
[[1769   17]
 [  70  721]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1786
           1       0.98      0.91      0.94       791

    accuracy                           0.97      2577
   macro avg       0.97      0.95      0.96      2577
weighted avg       0.97      0.97      0.97      2577



In [18]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(confusion_mat,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, predictions), recall_score(y_test, predictions),
                                         f1_score(y_test, predictions)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, predictions, pos_label=0)
recall_not_spam = recall_score(y_test, predictions, pos_label=0)
f1_score_not_spam = f1_score(y_test, predictions, pos_label=0)

precision_spam = precision_score(y_test, predictions, pos_label=1)
recall_spam = recall_score(y_test, predictions, pos_label=1)
f1_score_spam = f1_score(y_test, predictions, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))


comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

-------------------------------------------------------RANDOM FOREST-------------------------------------------------------

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['label'], test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_confusion_mat = confusion_matrix(y_test, rf_predictions)
rf_classification_rep = classification_report(y_test, rf_predictions)

print("Random Forest Classifier:")
print(f"Accuracy: {rf_accuracy}")
print("\nConfusion Matrix:")
print(rf_confusion_mat)
print("\nClassification Report:")
print(rf_classification_rep)


Random Forest Classifier:
Accuracy: 0.9875824602250679

Confusion Matrix:
[[1785    1]
 [  31  760]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1786
           1       1.00      0.96      0.98       791

    accuracy                           0.99      2577
   macro avg       0.99      0.98      0.99      2577
weighted avg       0.99      0.99      0.99      2577



In [20]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(rf_confusion_mat,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, rf_predictions), recall_score(y_test, rf_predictions),
                                         f1_score(y_test, rf_predictions)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, rf_predictions, pos_label=0)
recall_not_spam = recall_score(y_test, rf_predictions, pos_label=0)
f1_score_not_spam = f1_score(y_test, rf_predictions, pos_label=0)

precision_spam = precision_score(y_test, rf_predictions, pos_label=1)
recall_spam = recall_score(y_test, rf_predictions, pos_label=1)
f1_score_spam = f1_score(y_test, rf_predictions, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))


comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

-------------------------------------------------------SVM-------------------------------------------------------

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['label'], test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_confusion_mat = confusion_matrix(y_test, svm_predictions)
svm_classification_rep = classification_report(y_test, svm_predictions)

print("Support Vector Machine (SVM) Classifier:")
print(f"Accuracy: {svm_accuracy}")
print("\nConfusion Matrix:")
print(svm_confusion_mat)
print("\nClassification Report:")
print(svm_classification_rep)


Support Vector Machine (SVM) Classifier:
Accuracy: 0.989522700814901

Confusion Matrix:
[[1784    2]
 [  25  766]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1786
           1       1.00      0.97      0.98       791

    accuracy                           0.99      2577
   macro avg       0.99      0.98      0.99      2577
weighted avg       0.99      0.99      0.99      2577



In [23]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(svm_confusion_mat,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, svm_predictions), recall_score(y_test, svm_predictions),
                                         f1_score(y_test, svm_predictions)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, svm_predictions, pos_label=0)
recall_not_spam = recall_score(y_test, svm_predictions, pos_label=0)
f1_score_not_spam = f1_score(y_test, svm_predictions, pos_label=0)

precision_spam = precision_score(y_test, svm_predictions, pos_label=1)
recall_spam = recall_score(y_test, svm_predictions, pos_label=1)
f1_score_spam = f1_score(y_test, svm_predictions, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

-------------------------------------------------------DEEP LEARNING-------------------------------------------------------

In [ ]:
!pip install transformers
!pip install tensorflow
!pip install plotly

----------------------------------------------neural network-------------------------------------

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(
    tfidf_df, df['label'], test_size=0.2, random_state=42
)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 5
batch_size = 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

y_pred_prob = model.predict(X_test)
y_pred = np.round(y_pred_prob)

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Epoch 1/5
322/322 [==============================] - 4s 11ms/step - loss: 0.3340 - accuracy: 0.9080 - val_loss: 0.1412 - val_accuracy: 0.9569
Epoch 2/5
322/322 [==============================] - 3s 10ms/step - loss: 0.1222 - accuracy: 0.9638 - val_loss: 0.0892 - val_accuracy: 0.9763
Epoch 3/5
322/322 [==============================] - 3s 10ms/step - loss: 0.0784 - accuracy: 0.9757 - val_loss: 0.0708 - val_accuracy: 0.9818
Epoch 4/5
322/322 [==============================] - 4s 13ms/step - loss: 0.0516 - accuracy: 0.9868 - val_loss: 0.0610 - val_accuracy: 0.9837
Epoch 5/5
81/81 [==============================] - 0s 3ms/step - loss: 0.0568 - accuracy: 0.9853

Test Loss: 0.0568, Test Accuracy: 0.9853
81/81 [==============================] - 0s 3ms/step

Confusion Matrix:
[[1783    3]
 [  35  756]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1786
           1       1.00      0.96      0.98       791

    ac

In [25]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred), recall_score(y_test, y_pred),
                                         f1_score(y_test, y_pred)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred, pos_label=0)

precision_spam = precision_score(y_test, y_pred, pos_label=1)
recall_spam = recall_score(y_test, y_pred, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

-------------------------------------------------------N GRAM(bigram)-------------------------------------------------------

In [28]:
import pandas as pd
from nltk import ngrams
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def generate_ngrams(text, n):
    tokens = word_tokenize(text)
    n_grams = list(ngrams(tokens, n))
    return n_grams

def generate_ngrams_for_dataframe(dataframe, column_name, n):
    dataframe['ngrams'] = dataframe[column_name].apply(lambda x: generate_ngrams(x, n))
    return dataframe

df = pd.read_csv('stopwords_filtered.csv')
df['clean_comments'] = df['clean_comments'].fillna('')

n_value = 2

df_result = generate_ngrams_for_dataframe(df, 'clean_comments', n_value)
df_result


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,clean_comments,label,ngrams
0,bought meclay london thick dense shampoo amazi...,1,"[(bought, meclay), (meclay, london), (london, ..."
1,proud muslim,1,"[(proud, muslim)]"
2,proud muslim,1,"[(proud, muslim)]"
3,proud muslim,1,"[(proud, muslim)]"
4,proud muslim,1,"[(proud, muslim)]"
...,...,...,...
12876,may allah protect u always ameen safe journey,0,"[(may, allah), (allah, protect), (protect, u),..."
12877,wife cute n happy soull,0,"[(wife, cute), (cute, n), (n, happy), (happy, ..."
12878,love much shazada veer love jhelum punjab,0,"[(love, much), (much, shazada), (shazada, veer..."
12879,great work good see always,0,"[(great, work), (work, good), (good, see), (se..."


In [29]:
len(df_result['ngrams'])

12881

------------------naive baise classifier----------

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
def convert_ngrams_to_text(ngrams):
    return ' '.join([' '.join(gram) for gram in ngrams])

df_result['ngrams_text'] = df_result['ngrams'].apply(convert_ngrams_to_text)

X_train, X_test, y_train, y_test = train_test_split(
    df_result['ngrams_text'], df_result['label'], test_size=0.2, random_state=42
)

vectorizer = CountVectorizer()
X_train_matrix = vectorizer.fit_transform(X_train)
X_test_matrix = vectorizer.transform(X_test)

classifier = MultinomialNB()
classifier.fit(X_train_matrix, y_train)

y_pred = classifier.predict(X_test_matrix)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Accuracy: 0.9743888242142026
Confusion Matrix:
[[1743   43]
 [  23  768]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1786
           1       0.95      0.97      0.96       791

    accuracy                           0.97      2577
   macro avg       0.97      0.97      0.97      2577
weighted avg       0.97      0.97      0.97      2577



In [31]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred), recall_score(y_test, y_pred),
                                         f1_score(y_test, y_pred)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred, pos_label=0)

precision_spam = precision_score(y_test, y_pred, pos_label=1)
recall_spam = recall_score(y_test, y_pred, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

--------------------svm----------

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

def convert_ngrams_to_text(ngrams):
    return ' '.join([' '.join(gram) for gram in ngrams])

df_result['ngrams_text'] = df_result['ngrams'].apply(convert_ngrams_to_text)

X_train, X_test, y_train, y_test = train_test_split(
    df_result['ngrams_text'], df_result['label'], test_size=0.2, random_state=42
)

vectorizer = CountVectorizer()
X_train_matrix = vectorizer.fit_transform(X_train)
X_test_matrix = vectorizer.transform(X_test)

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_matrix, y_train)
y_pred_svm = svm_classifier.predict(X_test_matrix)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("\nSVM Confusion Matrix:")
print(conf_matrix_svm)
report_svm = classification_report(y_test, y_pred_svm)
print("\nSVM Classification Report:")
print(report_svm)


SVM Accuracy: 0.9941792782305006

SVM Confusion Matrix:
[[1784    2]
 [  13  778]]

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1786
           1       1.00      0.98      0.99       791

    accuracy                           0.99      2577
   macro avg       1.00      0.99      0.99      2577
weighted avg       0.99      0.99      0.99      2577



In [33]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix_svm,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred_svm), recall_score(y_test, y_pred_svm),
                                         f1_score(y_test, y_pred_svm)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred_svm, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred_svm, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred_svm, pos_label=0)

precision_spam = precision_score(y_test, y_pred_svm, pos_label=1)
recall_spam = recall_score(y_test, y_pred_svm, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred_svm, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

--------random forest------------

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

def convert_ngrams_to_text(ngrams):
    return ' '.join([' '.join(gram) for gram in ngrams])

df_result['ngrams_text'] = df_result['ngrams'].apply(convert_ngrams_to_text)

X_train, X_test, y_train, y_test = train_test_split(
    df_result['ngrams_text'], df_result['label'], test_size=0.2, random_state=42
)

vectorizer = CountVectorizer()
X_train_matrix = vectorizer.fit_transform(X_train)
X_test_matrix = vectorizer.transform(X_test)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_matrix, y_train)
y_pred_rf = rf_classifier.predict(X_test_matrix)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("\nRandom Forest Accuracy:", accuracy_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("\nRandom Forest Confusion Matrix:")
print(conf_matrix_rf)
report_rf = classification_report(y_test, y_pred_rf)
print("\nRandom Forest Classification Report:")
print(report_rf)



Random Forest Accuracy: 0.9868063639891347

Random Forest Confusion Matrix:
[[1785    1]
 [  33  758]]

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1786
           1       1.00      0.96      0.98       791

    accuracy                           0.99      2577
   macro avg       0.99      0.98      0.98      2577
weighted avg       0.99      0.99      0.99      2577



In [35]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix_rf,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred_rf), recall_score(y_test, y_pred_rf),
                                         f1_score(y_test, y_pred_rf)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred_rf, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred_rf, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred_rf, pos_label=0)

precision_spam = precision_score(y_test, y_pred_rf, pos_label=1)
recall_spam = recall_score(y_test, y_pred_rf, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred_rf, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

----------------------------------------------RNN(Bidirectional LSTM)-------------------------------------

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

df['ngrams'] = df['ngrams'].fillna('')

df['ngrams'] = df['ngrams'].apply(lambda x: ' '.join(x) if isinstance(x, tuple) else str(x))

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(
    df['ngrams'], df['label'], test_size=0.2, random_state=42
)

max_ngram_words = 10000
ngram_tokenizer = Tokenizer(num_words=max_ngram_words, oov_token='<OOV>')
ngram_tokenizer.fit_on_texts(X_train.apply(lambda x: str(x)))

X_train_ngram_sequences = ngram_tokenizer.texts_to_sequences(X_train.apply(lambda x: str(x)))
X_test_ngram_sequences = ngram_tokenizer.texts_to_sequences(X_test.apply(lambda x: str(x)))

max_ngram_length = 50
X_train_ngram_padded = pad_sequences(X_train_ngram_sequences, maxlen=max_ngram_length, padding='post', truncating='post')
X_test_ngram_padded = pad_sequences(X_test_ngram_sequences, maxlen=max_ngram_length, padding='post', truncating='post')

model = Sequential()
model.add(Embedding(input_dim=max_ngram_words, output_dim=100, input_length=max_ngram_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 5
batch_size = 32
model.fit(X_train_ngram_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_ngram_padded, y_test))

loss, accuracy = model.evaluate(X_test_ngram_padded, y_test)
print(f'\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

y_pred_prob = model.predict(X_test_ngram_padded)
y_pred = np.round(y_pred_prob)

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Epoch 1/5
322/322 [==============================] - 28s 76ms/step - loss: 0.1970 - accuracy: 0.9279 - val_loss: 0.0404 - val_accuracy: 0.9899
Epoch 2/5
322/322 [==============================] - 22s 69ms/step - loss: 0.0399 - accuracy: 0.9902 - val_loss: 0.0311 - val_accuracy: 0.9938
Epoch 3/5
322/322 [==============================] - 24s 73ms/step - loss: 0.0154 - accuracy: 0.9967 - val_loss: 0.0318 - val_accuracy: 0.9922
Epoch 4/5
322/322 [==============================] - 22s 69ms/step - loss: 0.0098 - accuracy: 0.9982 - val_loss: 0.0353 - val_accuracy: 0.9930
Epoch 5/5
81/81 [==============================] - 1s 15ms/step - loss: 0.0399 - accuracy: 0.9930

Test Loss: 0.0399, Test Accuracy: 0.9930
81/81 [==============================] - 2s 15ms/step

Confusion Matrix:
[[1780    6]
 [  12  779]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1786
           1       0.99      0.98      0.99       791



In [37]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred), recall_score(y_test, y_pred),
                                         f1_score(y_test, y_pred)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred, pos_label=0)

precision_spam = precision_score(y_test, y_pred, pos_label=1)
recall_spam = recall_score(y_test, y_pred, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()

GRU(Gated Recurrent Unit)

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

df['ngrams'] = df['ngrams'].fillna('')
df['ngrams'] = df['ngrams'].apply(lambda x: ' '.join(x) if isinstance(x, tuple) else str(x))

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(
    df['ngrams'], df['label'], test_size=0.2, random_state=42
)

max_ngram_words = 10000
ngram_tokenizer = Tokenizer(num_words=max_ngram_words, oov_token='<OOV>')
ngram_tokenizer.fit_on_texts(X_train.apply(lambda x: str(x)))

X_train_ngram_sequences = ngram_tokenizer.texts_to_sequences(X_train.apply(lambda x: str(x)))
X_test_ngram_sequences = ngram_tokenizer.texts_to_sequences(X_test.apply(lambda x: str(x)))

max_ngram_length = 50
X_train_ngram_padded = pad_sequences(X_train_ngram_sequences, maxlen=max_ngram_length, padding='post', truncating='post')
X_test_ngram_padded = pad_sequences(X_test_ngram_sequences, maxlen=max_ngram_length, padding='post', truncating='post')

model = Sequential()
model.add(Embedding(input_dim=max_ngram_words, output_dim=100, input_length=max_ngram_length))
model.add(Bidirectional(GRU(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 5
batch_size = 32
model.fit(X_train_ngram_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_ngram_padded, y_test))

loss, accuracy = model.evaluate(X_test_ngram_padded, y_test)
print(f'\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

y_pred_prob = model.predict(X_test_ngram_padded)
y_pred = np.round(y_pred_prob)

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)

Epoch 1/5
322/322 [==============================] - 26s 70ms/step - loss: 0.1620 - accuracy: 0.9422 - val_loss: 0.0583 - val_accuracy: 0.9907
Epoch 2/5
322/322 [==============================] - 20s 62ms/step - loss: 0.0552 - accuracy: 0.9908 - val_loss: 0.0466 - val_accuracy: 0.9907
Epoch 3/5
322/322 [==============================] - 21s 66ms/step - loss: 0.0195 - accuracy: 0.9962 - val_loss: 0.0488 - val_accuracy: 0.9911
Epoch 4/5
322/322 [==============================] - 19s 60ms/step - loss: 0.0131 - accuracy: 0.9980 - val_loss: 0.0424 - val_accuracy: 0.9919
Epoch 5/5
81/81 [==============================] - 2s 22ms/step - loss: 0.0434 - accuracy: 0.9915

Test Loss: 0.0434, Test Accuracy: 0.9915
81/81 [==============================] - 2s 13ms/step

Confusion Matrix:
[[1774   12]
 [  10  781]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1786
           1       0.98      0.99      0.99       791



In [39]:
import plotly.express as px
import plotly.graph_objects as go

accuracy_fig = go.Figure()
accuracy_fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Accuracy"},
    gauge={'axis': {'range': [0, 1]},
           'steps': [{'range': [0, 1], 'color': "lightgray"}],
           'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': accuracy}}))

accuracy_fig.show()

conf_matrix_fig = px.imshow(conf_matrix,
                            labels=dict(x="Predicted", y="True", color="Count"),
                            x=['Not Spam', 'Spam'],
                            y=['Not Spam', 'Spam'],
                            title='Confusion Matrix')
conf_matrix_fig.update_layout(autosize=False, width=500, height=400)

conf_matrix_fig.show()
classification_report_fig = px.imshow([[precision_score(y_test, y_pred), recall_score(y_test, y_pred),
                                         f1_score(y_test, y_pred)]],
                                       labels=dict(x="Metric", y="Value"),
                                       title='Classification Report Metrics',
                                       x=['Precision', 'Recall', 'F1-Score'],
                                       y=['Spam'])
classification_report_fig.update_layout(autosize=False, width=500, height=400)

classification_report_fig.show()
precision_not_spam = precision_score(y_test, y_pred, pos_label=0)
recall_not_spam = recall_score(y_test, y_pred, pos_label=0)
f1_score_not_spam = f1_score(y_test, y_pred, pos_label=0)

precision_spam = precision_score(y_test, y_pred, pos_label=1)
recall_spam = recall_score(y_test, y_pred, pos_label=1)
f1_score_spam = f1_score(y_test, y_pred, pos_label=1)

comparison_fig = go.Figure()

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_not_spam, recall_not_spam, f1_score_not_spam],
    name='Not Spam'
))

comparison_fig.add_trace(go.Bar(
    x=['Precision', 'Recall', 'F1-Score'],
    y=[precision_spam, recall_spam, f1_score_spam],
    name='Spam'
))

comparison_fig.update_layout(barmode='group', title='Comparison of Metrics for Spam and Not Spam Classes')

comparison_fig.show()